In [2]:
import torch
%load_ext autoreload
%autoreload 2

In [3]:
from model.threedepn import ThreeDEPNDecoder
from util.model import summarize_model

threedepn = ThreeDEPNDecoder()
print(summarize_model(threedepn))

   | Name         | Type             | Params  
-----------------------------------------------------
0  | bottleneck   | Sequential       | 197376  
1  | bottleneck.0 | Linear           | 65792   
2  | bottleneck.1 | ReLU             | 0       
3  | bottleneck.2 | Linear           | 131584  
4  | bottleneck.3 | ReLU             | 0       
5  | decoder1     | Sequential       | 8389376 
6  | decoder1.0   | ConvTranspose3d  | 8388864 
7  | decoder1.1   | BatchNorm3d      | 512     
8  | decoder1.2   | ReLU             | 0       
9  | decoder2     | Sequential       | 2097536 
10 | decoder2.0   | ConvTranspose3d  | 2097280 
11 | decoder2.1   | BatchNorm3d      | 256     
12 | decoder2.2   | ReLU             | 0       
13 | decoder3     | Sequential       | 524480  
14 | decoder3.0   | ConvTranspose3d  | 524352  
15 | decoder3.1   | BatchNorm3d      | 128     
16 | decoder3.2   | ReLU             | 0       
17 | decoder4     | Sequential       | 4097    
18 | decoder4.0   | ConvTranspose3

In [4]:
from data.shapenet import ShapeNet

# Create a dataset with train split
train_dataset = ShapeNet('train', filter_class='lamp_table')
val_dataset = ShapeNet('val', filter_class='lamp_table')
test_dataset = ShapeNet('test', filter_class='lamp_table')

print(f'Length of train set: {len(train_dataset)}')  # expected output: 153540
print(f'Length of val set: {len(val_dataset)}')  # expected output: 153540
print(f'Length of test set: {len(test_dataset)}')  # expected output: 64

Length of train set: 6654
Length of val set: 832
Length of test set: 832


In [5]:
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes

sample = train_dataset[0]
print(f'Target DF: {sample["target_df"].shape}')  # expected output: (32, 32, 32)
print(f'Target DF: {type(sample["target_df"])}')  # expected output: <class 'numpy.ndarray'>

input_mesh = marching_cubes(sample['target_df'], level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

Target DF: (32, 32, 32)
Target DF: <class 'numpy.ndarray'>


Output()

In [6]:
##################
#                #
#    TRAINING    #
#                #
##################

In [11]:
# OVERFIT
from scripts import train
config = {
    'experiment_name': 'overfit',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 32,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 4,
    'print_every_n': 100,
    'validate_every_n': 250,
    'latent_code_length' : 256,
    'scheduler_step_size': 20,
    'vad_free' : True,
    'test': False,
    'kl_weight': 1,
    'resume_ckpt': None,
    'filter_class': 'car_airplane',
    'decoder_var' : True

}
train.main(config)

Using device: cuda:0
Data length 9994
Training params: 3
[000/00099] train_loss: 0.288746 kl_loss: 0.166894 normal_loss: 0.121852
[000/00199] train_loss: 0.225641 kl_loss: 0.166353 normal_loss: 0.059288
[000/00299] train_loss: 0.219455 kl_loss: 0.166727 normal_loss: 0.052728
[001/00086] train_loss: 0.176847 kl_loss: 0.128196 normal_loss: 0.048651
[001/00186] train_loss: 0.170828 kl_loss: 0.122256 normal_loss: 0.048573
[001/00286] train_loss: 0.169678 kl_loss: 0.122033 normal_loss: 0.047645
[002/00073] train_loss: 0.142393 kl_loss: 0.094651 normal_loss: 0.047742
[002/00173] train_loss: 0.130873 kl_loss: 0.084667 normal_loss: 0.046206


KeyboardInterrupt: 

In [ ]:
# AIRPLANE VAD
from scripts import train
config = {
    'experiment_name': 'airplane_vad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 20,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'resume_ckpt': 'airplane_vad',
    'filter_class': 'airplane',
    'decoder_var' : True
}
train.main(config)

In [ ]:
# AIRPLANE AD
from scripts import train
config = {
    'experiment_name': 'airplane_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 20,
    'vad_free' : False,
    'test': False,
    'kl_weight': 0.01,
    'resume_ckpt': None,
    'filter_class': 'airplane',
    'decoder_var' : False
}
train.main(config)

In [ ]:
# CHAIR
from scripts import train
config = {
    'experiment_name': 'chair_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.01,
    'resume_ckpt': None,
    'filter_class': 'chair',
    'decoder_var' : False
}
train.main(config)

In [ ]:
# SOFA & CHAIR VAD
from scripts import train
config = {
    'experiment_name': 'sofa_chair_vad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'resume_ckpt': None,
    'filter_class': 'sofa_chair',
    'decoder_var' : True
}
train.main(config)

In [ ]:
# SOFA VAD
from scripts import train
config = {
    'experiment_name': 'sofa_vad_0_03kl',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'resume_ckpt': None,
    'filter_class': 'sofa',
    'decoder_var' : True
}
train.main(config)

In [ ]:
# SOFA AD
from scripts import train
config = {
    'experiment_name': 'sofa_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : False,
    'test': False,
    'kl_weight': 0.01,
    'resume_ckpt': None,
    'filter_class': 'sofa',
    'decoder_var' : False
}
train.main(config)

In [ ]:
# SOFA VAD
from scripts import train
config = {
    'experiment_name': 'car_vad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'resume_ckpt': None,
    'filter_class': 'car',
    'decoder_var' : True
}
train.main(config)

In [ ]:
# CAR AD
from scripts import train
config = {
    'experiment_name': 'car_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : False,
    'test': False,
    'kl_weight': 0.0,
    'resume_ckpt': None,
    'filter_class': 'car',
    'decoder_var' : False
}
train.main(config)

In [ ]:
# CAR AIRPLANE AD
from scripts import train
config = {
    'experiment_name': 'car_airplane_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : False,
    'test': False,
    'kl_weight': 0.0,
    'resume_ckpt': 'car_airplane_ad',
    'filter_class': 'car_airplane',
    'decoder_var' : False
}
train.main(config)

In [ ]:
#####################
#                   #
#    VISUALIZING    #
#                   #
#####################

In [41]:
from scripts.visualize import visualize_dataset_sample, visualize_ad, visualize_vad, visualize_vad_norm, visualize_vad_norm

In [42]:
import random
experiment = "sofa_vad"
experiment2 = "sofa_ad"
# experiment2 = "sofa_ad"
filter_class = "car_airplane"
index = 4123
index1 = random.choice(range(len(ShapeNet('train', filter_class = filter_class))))
index2 = random.choice(range(len(ShapeNet('train', filter_class = filter_class))))
a1 = 0.5
a2 = 1 - a1
#-------
# visualize_vad_norm(experiment2)
visualize_vad_norm(experiment)
visualize_vad_norm(experiment2)
# visualize_ad(experiment, index)
#-------
# visualize_vad_norm(experiment)
# visualize_vad_norm(experiment2)
# visualize_ad(experiment2, index)
# visualize_dataset_sample(filter_class, index)
#-------
# visualize_interpolation_ad(experiment, index1, index2, a1, a2)
# visualize_ad(experiment, index1)
# visualize_ad(experiment, index2)

Output()

Output()

In [6]:
####################
#                  #
#    EVALUATION    #
#                  #
####################

In [77]:
from scripts.evaluate import mmd, generate_samples, convert_df_to_point_cloud, visualize_point_cloud, chamfer_distance_numpy, mmd
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes
import numpy as np

In [85]:
# MMD

# generate n new samples
n = 5
samples = generate_samples('sofa_vad', n)

sample1 = samples[0]
sample2 = samples[1]

# visualize
input_mesh = marching_cubes(sample1.squeeze(0).detach().numpy(), level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

input_mesh = marching_cubes(sample2.squeeze(0).detach().numpy(), level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

point_cloud = convert_df_to_point_cloud(sample1.squeeze(0))
visualize_point_cloud(point_cloud)

point_cloud2 = convert_df_to_point_cloud(sample2.squeeze(0))
visualize_point_cloud(point_cloud2)

Output()

Output()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [90]:
# calculate chamfer distance between the 2 samples
chamfer_distance_numpy(point_cloud, point_cloud2)

1.7323260368599513

In [91]:
# calculate mmd between the 2 samples
# Note: the final value changes a bit every time because every time a new point cloud is generated
# Note: mmd function takes a lot of time, needs optimization
mmd(samples, samples)

array([1.55289372, 1.37301004, 1.41850447, 1.62323834, 1.83847967])